In [63]:
import os, sys
import pandas as pd
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import datetime
from datetime import date
%matplotlib inline

In [94]:
# data_dir = '/Users/honggao/Downloads/2016-06/'
# data_filedir = data_dir + 'bikeshare_nyc_raw.csv'

In [115]:
data_dir = '/Users/honggao/Downloads/'
data_filedir = data_dir + '2016-01.csv'

In [116]:
data = pd.read_table(data_filedir, sep='\t', error_bad_lines=False)
data.head()

b'Skipping line 596959: expected 14 fields, saw 26\n'


,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6
0,72,W 52 St & 11 Ave,16-01-01,1,12,0,10,27,39,40.767272,-73.993929,1,1,NaN
1,72,W 52 St & 11 Ave,16-01-01,1,24,0,10,27,39,40.767272,-73.993929,1,1,NaN
2,72,W 52 St & 11 Ave,16-01-01,1,35,0,10,27,39,40.767272,-73.993929,1,1,NaN
3,72,W 52 St & 11 Ave,16-01-01,1,46,0,10,27,39,40.767272,-73.993929,1,1,NaN
4,72,W 52 St & 11 Ave,16-01-01,1,57,0,9,28,39,40.767272,-73.993929,1,1,NaN


In [117]:
data.describe()

,dock_id,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6
count,1.597490e+06,1.597490e+06,1.597490e+06,1.597490e+06,1.597490e+06,1.597490e+06,1.597490e+06,1.597490e+06,1.597490e+06,1.597490e+06,1.597490e+06,0.0
mean,1.436694e+03,6.500661e+00,2.926616e+01,4.802935e-01,9.717199e+00,2.077906e+01,3.269603e+01,4.072857e+01,-7.398346e+01,8.398782e-01,1.316965e+00,NaN
std,1.332438e+03,3.480518e+00,1.735927e+01,4.996117e-01,1.035028e+01,1.307785e+01,1.143434e+01,3.192459e-01,6.499218e-02,3.667191e-01,7.303856e-01,NaN
min,7.200000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-7.409694e+01,0.000000e+00,1.000000e+00,NaN
25%,3.460000e+02,3.000000e+00,1.400000e+01,0.000000e+00,1.000000e+00,1.100000e+01,2.400000e+01,4.070877e+01,-7.399704e+01,1.000000e+00,1.000000e+00,NaN
50%,4.880000e+02,7.000000e+00,2.900000e+01,0.000000e+00,7.000000e+00,1.900000e+01,3.100000e+01,4.072560e+01,-7.398261e+01,1.000000e+00,1.000000e+00,NaN
75%,3.102000e+03,1.000000e+01,4.400000e+01,1.000000e+00,1.500000e+01,2.900000e+01,3.900000e+01,4.074916e+01,-7.396264e+01,1.000000e+00,1.000000e+00,NaN
max,3.244000e+03,1.200000e+01,5.900000e+01,1.000000e+00,6.100000e+01,6.700000e+01,6.700000e+01,4.407427e+02,0.000000e+00,1.000000e+00,3.000000e+00,NaN


In [118]:
data.shape

(1597490, 14)

In [119]:
data.dtypes

dock_id          int64
dock_name       object
date            object
hour             int64
minute           int64
pm               int64
avail_bikes      int64
avail_docks      int64
tot_docks        int64
_lat           float64
_long          float64
in_service       int64
status_key       int64
dock_name6     float64
dtype: object

In [120]:
# keep rush hours 6-10am and 4-8pm
data_morning = data[(data['pm'] == 0) & (data['hour'].isin(range(6,11)))]
data_morning.head()

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6
27,72,W 52 St & 11 Ave,16-01-01,6,10,0,8,29,39,40.767272,-73.993929,1,1,NaN
28,72,W 52 St & 11 Ave,16-01-01,6,21,0,8,29,39,40.767272,-73.993929,1,1,NaN
29,72,W 52 St & 11 Ave,16-01-01,6,33,0,8,29,39,40.767272,-73.993929,1,1,NaN
30,72,W 52 St & 11 Ave,16-01-01,6,44,0,8,29,39,40.767272,-73.993929,1,1,NaN
31,72,W 52 St & 11 Ave,16-01-01,6,55,0,8,29,39,40.767272,-73.993929,1,1,NaN


In [121]:
data_afternoon = data[(data['pm'] == 1) & (data['hour'].isin(range(4,9)))]
data_afternoon.head()

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6
73,72,W 52 St & 11 Ave,16-01-01,4,1,1,3,34,39,40.767272,-73.993929,1,1,NaN
74,72,W 52 St & 11 Ave,16-01-01,4,14,1,3,34,39,40.767272,-73.993929,1,1,NaN
75,72,W 52 St & 11 Ave,16-01-01,4,26,1,3,34,39,40.767272,-73.993929,1,1,NaN
76,72,W 52 St & 11 Ave,16-01-01,4,40,1,3,34,39,40.767272,-73.993929,1,1,NaN
77,72,W 52 St & 11 Ave,16-01-01,4,52,1,3,34,39,40.767272,-73.993929,1,1,NaN


In [122]:
data = pd.concat([data_morning, data_afternoon])

In [123]:
data['shortage'] = [1 if x<=5 else 0 for x in data['avail_bikes']]

In [124]:
data.head()

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6,shortage
27,72,W 52 St & 11 Ave,16-01-01,6,10,0,8,29,39,40.767272,-73.993929,1,1,NaN,0
28,72,W 52 St & 11 Ave,16-01-01,6,21,0,8,29,39,40.767272,-73.993929,1,1,NaN,0
29,72,W 52 St & 11 Ave,16-01-01,6,33,0,8,29,39,40.767272,-73.993929,1,1,NaN,0
30,72,W 52 St & 11 Ave,16-01-01,6,44,0,8,29,39,40.767272,-73.993929,1,1,NaN,0
31,72,W 52 St & 11 Ave,16-01-01,6,55,0,8,29,39,40.767272,-73.993929,1,1,NaN,0


In [125]:
data['datetime'] = pd.to_datetime(data['date'], format='%y-%m-%d')
data.head()

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6,shortage,datetime
27,72,W 52 St & 11 Ave,16-01-01,6,10,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01
28,72,W 52 St & 11 Ave,16-01-01,6,21,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01
29,72,W 52 St & 11 Ave,16-01-01,6,33,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01
30,72,W 52 St & 11 Ave,16-01-01,6,44,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01
31,72,W 52 St & 11 Ave,16-01-01,6,55,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01


In [126]:
data.dtypes

dock_id                 int64
dock_name              object
date                   object
hour                    int64
minute                  int64
pm                      int64
avail_bikes             int64
avail_docks             int64
tot_docks               int64
_lat                  float64
_long                 float64
in_service              int64
status_key              int64
dock_name6            float64
shortage                int64
datetime       datetime64[ns]
dtype: object

In [127]:
data['day_of_week'] = data['datetime'].dt.weekday_name
data.head()

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6,shortage,datetime,day_of_week
27,72,W 52 St & 11 Ave,16-01-01,6,10,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday
28,72,W 52 St & 11 Ave,16-01-01,6,21,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday
29,72,W 52 St & 11 Ave,16-01-01,6,33,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday
30,72,W 52 St & 11 Ave,16-01-01,6,44,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday
31,72,W 52 St & 11 Ave,16-01-01,6,55,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday


In [128]:
# days = {'weekday':['Monday', 'Tuesday', 'Wednesday', 'Thursday'],'weekend':['Friday', 'Saturday', 'Sunday']}

data['weekday'] = [0 if x in ['Saturday','Sunday'] else 1 for x in data['day_of_week']]

In [129]:
data.head()

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6,shortage,datetime,day_of_week,weekday
27,72,W 52 St & 11 Ave,16-01-01,6,10,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1
28,72,W 52 St & 11 Ave,16-01-01,6,21,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1
29,72,W 52 St & 11 Ave,16-01-01,6,33,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1
30,72,W 52 St & 11 Ave,16-01-01,6,44,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1
31,72,W 52 St & 11 Ave,16-01-01,6,55,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1


In [130]:
holiday = ['2016-01-01','2016-01-18', '2016-02-15', '2016-05-30',
           '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-25']

In [131]:
holiday = pd.to_datetime(holiday, format='%Y-%m-%d')
holiday

DatetimeIndex(['2016-01-01', '2016-01-18', '2016-02-15', '2016-05-30',
               '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11',
               '2016-11-24', '2016-12-25'],
              dtype='datetime64[ns]', freq=None)

In [132]:
data['holiday'] = [1 if x in holiday else 0 for x in data['datetime']]

In [133]:
data.head()

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6,shortage,datetime,day_of_week,weekday,holiday
27,72,W 52 St & 11 Ave,16-01-01,6,10,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1,1
28,72,W 52 St & 11 Ave,16-01-01,6,21,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1,1
29,72,W 52 St & 11 Ave,16-01-01,6,33,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1,1
30,72,W 52 St & 11 Ave,16-01-01,6,44,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1,1
31,72,W 52 St & 11 Ave,16-01-01,6,55,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1,1


In [135]:
data.holiday.describe()

count    640191.000000
mean          0.847074
std           0.359917
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: in_service, dtype: float64

In [138]:
data[(data['in_service'] == 0) & (data['avail_bikes'] > 0)].count()

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6,shortage,datetime,day_of_week,weekday,holiday
53475,153,E 40 St & 5 Ave,16-01-01,6,10,0,1,28,29,40.752062,-73.981632,0,3,NaN,1,2016-01-01,Friday,1,1
53476,153,E 40 St & 5 Ave,16-01-01,6,21,0,1,28,29,40.752062,-73.981632,0,3,NaN,1,2016-01-01,Friday,1,1
53477,153,E 40 St & 5 Ave,16-01-01,6,33,0,1,28,29,40.752062,-73.981632,0,3,NaN,1,2016-01-01,Friday,1,1
53478,153,E 40 St & 5 Ave,16-01-01,6,44,0,1,28,29,40.752062,-73.981632,0,3,NaN,1,2016-01-01,Friday,1,1
53479,153,E 40 St & 5 Ave,16-01-01,6,55,0,1,28,29,40.752062,-73.981632,0,3,NaN,1,2016-01-01,Friday,1,1
53480,153,E 40 St & 5 Ave,16-01-01,7,7,0,1,28,29,40.752062,-73.981632,0,3,NaN,1,2016-01-01,Friday,1,1
53481,153,E 40 St & 5 Ave,16-01-01,7,19,0,1,28,29,40.752062,-73.981632,0,3,NaN,1,2016-01-01,Friday,1,1
53482,153,E 40 St & 5 Ave,16-01-01,7,31,0,1,28,29,40.752062,-73.981632,0,3,NaN,1,2016-01-01,Friday,1,1
53483,153,E 40 St & 5 Ave,16-01-01,7,43,0,1,28,29,40.752062,-73.981632,0,3,NaN,1,2016-01-01,Friday,1,1
53484,153,E 40 St & 5 Ave,16-01-01,7,55,0,1,28,29,40.752062,-73.981632,0,3,NaN,1,2016-01-01,Friday,1,1


In [139]:
data[data['in_service'] == 0].count()

dock_id        97902
dock_name      97902
date           97902
hour           97902
minute         97902
pm             97902
avail_bikes    97902
avail_docks    97902
tot_docks      97902
_lat           97902
_long          97902
in_service     97902
status_key     97902
dock_name6         0
shortage       97902
datetime       97902
day_of_week    97902
weekday        97902
holiday        97902
dtype: int64

In [151]:
data['minute_bin'] = pd.cut(data['minute'], bins=[0, 16, 31, 46, 59], include_lowest=True, labels=['1', '2', '3', '4'])
data.head()

,dock_id,dock_name,date,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_name6,shortage,datetime,day_of_week,weekday,holiday,minute_bin
27,72,W 52 St & 11 Ave,16-01-01,6,10,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1,1,1
28,72,W 52 St & 11 Ave,16-01-01,6,21,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1,1,2
29,72,W 52 St & 11 Ave,16-01-01,6,33,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1,1,3
30,72,W 52 St & 11 Ave,16-01-01,6,44,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1,1,3
31,72,W 52 St & 11 Ave,16-01-01,6,55,0,8,29,39,40.767272,-73.993929,1,1,NaN,0,2016-01-01,Friday,1,1,4


In [ ]:
# Next step
# For rows in each minute_bin, keep one instance with lower number of avail_bikes. 